In [5]:
import sqlite3
conn = sqlite3.connect('chinook1.db')
import pandas as pd

# SQLITE

## Connect to Sqlite db

In [2]:
import sqlite3
conn = sqlite3.connect('chinook1.db')
import pandas as pd

## Connect to MySQL db

In [3]:
# show table name
query = '''
SELECT name
FROM sqlite_master 
    WHERE type ='table' 
      AND name NOT LIKE 'sqlite_%'
'''

pd.read_sql_query(query, conn)

,name
0,albums
1,artists
2,customers
3,employees
4,genres
5,invoices
6,invoice_items
7,media_types
8,playlists
9,playlist_track


# SCHEMA

## Create Schema

## Create Table

## Alter Table

In [10]:
# Find all columns not having any value
SELECT column_name, COUNT(1) AS nulls_count, (SELECT COUNT(*) FROM `table_name`) AS rows_count
FROM `table_name`, UNNEST(REGEXP_EXTRACT_ALL(TO_JSON_STRING(`table_name`), r'"(\w+)":null')) column_name 
GROUP BY column_name 
HAVING nulls_count = rows_count
ORDER BY nulls_count DESC

SyntaxError: invalid syntax (2219221705.py, line 2)

## Drop 

# QUERY

In [7]:
pd.read_sql_query('SELECT FirstName,LastName FROM Employees', conn)

,FirstName,LastName
0,Andrew,Adams
1,Nancy,Edwards
2,Jane,Peacock
3,Margaret,Park
4,Steve,Johnson
5,Michael,Mitchell
6,Robert,King
7,Laura,Callahan


## Filter

In [ ]:
# CASE WHEN

In [ ]:
# CASE EXCLUDE
SELECT *
FROM table
WHERE 
    CASE WHEN a > 5 THEN 'Keep'
         WHEN a <= 5 THEN 'Exclude' END = 'Keep';

# WINDOW FUNCTION

## Aggregate SUM(), MAX(), MIN(), AVG(), COUNT()

In [8]:
query = """
SELECT artists.artistID, COUNT(albums.albumID) 
FROM artists JOIN albums ON artists.artistID = albums.artistID
GROUP BY artists.artistID
"""

pd.read_sql_query(query, conn)

,ArtistId,COUNT(albums.albumID)
0,1,2
1,2,2
2,3,1
3,4,1
4,5,1
...,...,...
199,271,1
200,272,1
201,273,1
202,274,1


In [11]:
%%capture
!curl https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release | tar xz
%cd sqlite/
!./configure
!make sqlite3.c
%cd /content
!npx degit coleifer/pysqlite3 -f
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build
!cp build/lib.linux-x86_64-3.7/pysqlite3/_sqlite3.cpython-37m-x86_64-linux-gnu.so \
     /usr/lib/python3.7/lib-dynload/

In [1]:
# then MENU: Runtime > Restart runtime ...
import sqlite3
print(sqlite3.sqlite_version) 

conn = sqlite3.connect('chinook.db')
import pandas as pd

3.37.2


In [10]:
# Displays the artist's name, the names of their albums, and the total number of albums they released
query = """
SELECT artists.Name, albums.Title, COUNT(albums.albumID) OVER(PARTITION BY artists.artistID) AS Number_of_albums
FROM artists JOIN albums ON artists.artistID = albums.artistID
"""

pd.read_sql_query(query, conn)
# We can't show the information of all albums of an artists using GROUP BY + AGGREGATE FUNCTIONS

,Name,Title,Number_of_albums
0,AC/DC,For Those About To Rock We Salute You,2
1,AC/DC,Let There Be Rock,2
2,Accept,Balls to the Wall,2
3,Accept,Restless and Wild,2
4,Aerosmith,Big Ones,1
...,...,...,...
342,"Mela Tenenbaum, Pro Musica Prague & Richard Kapp","Locatelli: Concertos for Violin, Strings and C...",1
343,Emerson String Quartet,Schubert: The Late String Quartets & String Qu...,1
344,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon...",Monteverdi: L'Orfeo,1
345,Nash Ensemble,Mozart: Chamber Music,1


## Ranking RANK(), DENSE_RANK(), ROW_NUMBER(), NTILE()

In [9]:
# From the number of albums per artist. Let's create a ranking column based on the number of albums that the artist released. Each line is an artist
# # Create a temporary table with information about the number of albums for each artist
# # Based on that temp table create 1 new column for ranking
query = """
WITH tmp AS
(
  SELECT artists.artistID,artists.Name,  COUNT(albums.albumID) AS Number_of_albums
  FROM artists JOIN albums ON artists.artistID = albums.artistID
  GROUP BY artists.artistID, artists.Name
)

SELECT *, 
  DENSE_RANK() OVER (ORDER BY tmp.Number_of_albums DESC) AS Rank
FROM tmp

"""

pd.read_sql_query(query, conn)

,artistID,Name,Number_of_albums,Rank
0,90,Iron Maiden,21,1
1,22,Led Zeppelin,14,2
2,58,Deep Purple,11,3
3,50,Metallica,10,4
4,150,U2,10,4
...,...,...,...,...
199,271,"Mela Tenenbaum, Pro Musica Prague & Richard Kapp",1,10
200,272,Emerson String Quartet,1,10
201,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon...",1,10
202,274,Nash Ensemble,1,10


In [ ]:
#  Ranked by track length


## Value LAG(), LEAD(), FIRST_VALUE(), LAST_VALUE()